In [7]:
import ppigrf
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime

https://www.ngdc.noaa.gov/geomag/calculators/magcalc.shtml#igrfwmm

In [6]:
data = pd.read_csv("data/tag_tracks.csv").rename({
    "Ptt": "ptt",
    "Latitude": "lat",
    "Longitude": "lon",
    "Dates - Date Key → Date": "date",
    "Dates - Date Key → Year": "year",
    "Dates - Date Key → Month": "month",
    "Dates - Date Key → Day": "day",
}, axis=1)
data['date'] = pd.to_datetime(data['date'])
print(data.shape)
data.head()

(7532, 7)


,ptt,lat,lon,date,year,month,day
0,129843,54.131760,-166.922615,2013-12-19,2013,12,19
1,129843,54.258072,-166.884086,2013-12-20,2013,12,20
2,129843,54.312433,-166.910525,2013-12-21,2013,12,21
3,129843,54.358280,-166.817057,2013-12-22,2013,12,22
4,129843,54.389694,-166.676901,2013-12-23,2013,12,23


In [46]:
data.sample(5).apply(lambda row: [x[0][0] for x in ppigrf.igrf(row['lon'], row['lat'], 0, row['date'])], axis=1)

2399     3644.602806
2399    19779.627145
2399   -47175.590489
1527     4138.781648
1527      19268.1004
1527   -47818.047832
943      4403.921895
943     15903.641097
943    -52113.409926
791      4463.851384
791     15784.864046
791    -52287.266309
3936     5219.833866
3936    16005.387137
3936   -51974.167041
dtype: object

In [47]:
rows = []
for _, row in tqdm(data.iterrows()):
    EnT, NnT, ZnT = [x[0][0] for x in ppigrf.igrf(row['lon'], row['lat'], 0, row['date'])]
    rows.append({
        "ptt": row['ptt'],
        "date": row['date'],
        "EnT": EnT,
        "NnT": NnT,
        "ZnT": ZnT,
    })

df = pd.DataFrame(rows)
df.head()

7532it [01:24, 88.85it/s]


,ptt,date,EnT,NnT,ZnT
0,129843,2013-12-19,3484.718376,20199.007250,-46678.421745
1,129843,2013-12-20,3482.529201,20139.301976,-46785.348153
2,129843,2013-12-21,3475.198232,20117.118268,-46827.370407
3,129843,2013-12-22,3485.772678,20088.540981,-46873.438963
4,129843,2013-12-23,3503.925882,20062.429516,-46912.088257


In [50]:
data = data[[c for c in data.columns if c not in ["EnT", "NnT", "ZnT"]]].merge(
    df, on=["ptt", "date"], how="inner"
)
data['nT'] = np.sqrt(data['EnT']**2 + data['NnT']**2 + data['ZnT']**2)
data['inclination'] = np.degrees(np.arctan2(data['ZnT'], np.sqrt(data['EnT']**2 + data['NnT']**2)))
data.head()

,ptt,lat,lon,date,year,month,day,nT,EnT,NnT,ZnT,inclination
0,129843,54.131760,-166.922615,2013-12-19,2013,12,19,50980.567010,3484.718376,20199.007250,-46678.421745,-66.292797
1,129843,54.258072,-166.884086,2013-12-20,2013,12,20,51054.757815,3482.529201,20139.301976,-46785.348153,-66.401881
2,129843,54.312433,-166.910525,2013-12-21,2013,12,21,51084.029495,3475.198232,20117.118268,-46827.370407,-66.444499
3,129843,54.358280,-166.817057,2013-12-22,2013,12,22,51115.744836,3485.772678,20088.540981,-46873.438963,-66.492215
4,129843,54.389694,-166.676901,2013-12-23,2013,12,23,51142.180236,3503.925882,20062.429516,-46912.088257,-66.532716


In [18]:
def plot_it(data, lat, lon, color):
    fig = px.scatter_geo(
        data, lat=lat, lon=lon, color=color,
    )
    fig.update_layout(autosize=True, height=600, geo=dict(center=dict(lat=58, lon=-150), projection_scale=6))
    return fig

plot_it(data, "lat", "lon", "ptt").show()

In [55]:
from tqdm import tqdm

rows = []
lon_scale = 0.5
lat_scale = 0.5
for lon in tqdm(np.arange(data['lon'].min(), data['lon'].max(), lon_scale)):
    for lat in np.arange(data['lat'].min(), data['lat'].max(), lat_scale):
        EnT, NnT, ZnT = [x[0][0] for x in ppigrf.igrf(lon, lat, 0, datetime(2019, 1, 1))]
        rows.append({
            'lat': lat, 
            'lon': lon, 
            'EnT': EnT,
            'NnT': NnT,
            'ZnT': ZnT,
            'nT': np.sqrt(EnT**2 + NnT**2 + ZnT**2),
            'inclination': np.degrees(np.arctan2(ZnT, np.sqrt(EnT**2 + NnT**2))),
        })

df = pd.DataFrame(rows)


100%|██████████| 113/113 [00:39<00:00,  2.84it/s]


In [56]:
df['nT_discrete'] = df['nT'] - df['nT'] % 500
df['nT_discrete'] = df['nT_discrete'].astype(str)
plot_it(df, "lat", "lon", "nT_discrete").show()

In [57]:
df['inclination_discrete'] = df['inclination'] - df['inclination'] % 1
df['inclination_discrete'] = df['inclination_discrete'].astype(str)
plot_it(df, "lat", "lon", "inclination_discrete").show()

In [34]:
plot_it(data, "lat", "lon", "ptt").show()

In [65]:
ptt = '159017b'
plot_it(data[data['ptt'] == ptt].sort_values('date', ascending=True).reset_index(drop=True).reset_index(), "lat", "lon", "index").show()

In [66]:
plot_it(data[data['ptt'] == ptt], "lat", "lon", "inclination").show()

In [67]:
plot_it(data[data['ptt'] == ptt], "lat", "lon", "nT").show()

In [64]:
plot_it(data[data['month'].isin([5, 6, 7])], "lat", "lon", "ptt").show()

In [ ]:
ptt = '159017b'